In [1]:
import torch
import os
from pathlib import Path
#from logs.locallogger import LocalLogger3D
from logs.wandblogger import WandBLogger3D 
from training.trainer import MRTrainer
from datasets.signals import VolumeSignal, Procedural3DSignal
from datasets.utils import checker
from networks.mrnet import MRFactory
from datasets.pyramids import create_MR_structure
import yaml
from yaml.loader import SafeLoader
import matplotlib.pyplot as plt
from datasets.procedural import voronoi_texture

In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "train3d.ipynb"
BASE_DIR = Path('.').absolute().parents[0]
VOXEL_PATH = BASE_DIR.joinpath('vox')
MODEL_PATH = BASE_DIR.joinpath('models')

In [3]:
project_name = "dev-sandbox"
config_file = '../configs/config_3d_m_net.yml'
with open(config_file) as f:
    hyper = yaml.load(f, Loader=SafeLoader)
    if isinstance(hyper['batch_size'], str):
        hyper['batch_size'] = eval(hyper['batch_size'])
    print(hyper)
filepath = os.path.join(VOXEL_PATH, hyper['filename'])

{'model': 'M', 'in_features': 3, 'out_features': 3, 'hidden_layers': 1, 'hidden_features': [[256, 256], [512, 256], [512, 512]], 'bias': True, 'max_stages': 1, 'period': 2, 'domain': [-1, 1], 'omega_0': [4, 12, 24], 'hidden_omega_0': [30, 30, 30, 30], 'superposition_w0': False, 'sampling_scheme': 'regular', 'decimation': True, 'filter': 'gauss', 'attributes': ['d0'], 'loss_function': 'mse', 'loss_weights': {'d0': 1, 'd1': 0}, 'opt_method': 'Adam', 'lr': 0.001, 'loss_tol': 1e-12, 'diff_tol': 1e-09, 'max_epochs_per_stage': [2, 150, 100, 5], 'batch_size': 131072, 'filename': 'voronoi', 'width': 64, 'height': 64, 'depth': 64, 'channels': 3, 'test_mesh': 'bunny.obj', 'ntestpoints': 290000, 'device': 'cuda', 'eval_device': 'cpu', 'save_format': 'general', 'visualize_grad': True, 'extrapolate': [-2, 2], 'slice_views': ['x', 'y', 'z', 'xy'], 'join_views': False}


In [4]:
hyper['batch_size']

131072

In [5]:
# vol = torch.from_numpy(checker(dim, 32))
# base_signal = VolumeSignal(vol.view((1, dim, dim, dim)),
#                            hyper['domain'],
#                            batch_size=hyper['batch_size'])
base_signal = VolumeSignal.init_fromfile(filepath,
                                         hyper['domain'],
                                         channels=hyper['channels'],
                                         batch_size=hyper['batch_size'])

train_dataloader = create_MR_structure(base_signal, hyper['max_stages'],
                                        hyper['filter'], hyper['decimation'])
test_dataloader = create_MR_structure(base_signal, hyper['max_stages'],
                                        hyper['filter'], False)
# dim = hyper['width']
# proc = voronoi_texture(4)
# base_signal = Procedural3DSignal(
#     proc,
#     (dim, dim, dim),
#     channels=hyper['channels'],
#     domain=hyper['domain'],
#     batch_size=hyper['batch_size']
# )
# train_dataloader = [base_signal]
# test_dataloader = [base_signal]

In [6]:
filename = os.path.basename(hyper['filename'])
wandblogger = WandBLogger3D(project_name,
                            f"{hyper['model']}{hyper['filter'][0].upper()}{filename[0:5]}",
                            hyper,
                            BASE_DIR)
mrmodel = MRFactory.from_dict(hyper)
print("Model: ", type(mrmodel))
mrtrainer = MRTrainer.init_from_dict(mrmodel, train_dataloader, test_dataloader, wandblogger, hyper)
mrtrainer.train(hyper['device'])

Model:  <class 'networks.mrnet.MNet'>


wandb: Currently logged in as: hallpaz (siren-song). Use `wandb login --relogin` to force relogin


DATA SIZE (3, 64, 64, 64)
[Logger] All inference done in 7.359356164932251s on cpu
File  MGvoron_1-1_w4F_hf256256_MEp2_hl1_r64_pr2.pth


D0 loss,▁█
D0 loss,0.19559


Total model parameters =  67586
Training finished after 2 epochs
